# [실습 4] RAG(Retrieval-Augmented Generation) Tool 만들기

학습 내용:
1. 환경 설정: RAG에 필요한 라이브러리 설치
2. 데이터 로드: "Attention Is All You Need" 논문(PDF) 로드
3. 텍스트 전처리: 로드한 문서를 의미 있는 단위(Chunk)로 분할
4. 임베딩 및 벡터 스토어 구축:
   - HuggingFace의 임베딩 모델 사용
   - 분할된 텍스트를 벡터로 변환하여 FAISS 벡터 스토어에 저장
5. RAG Tool 생성: 벡터 스토어에서 관련 문서를 검색하는 Retriever Tool 구현
6. Agent 연동: RAG Tool과 웹 검색 Tool을 함께 사용하는 Agent 생성 및 테스트

```
# 사용 모델:
# - LLM: gpt-4o-mini
# - Embedding: all-MiniLM-L6-v2 (HuggingFace)
```

## 1, 환경 설정

In [ ]:
import os

from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## 2. 데이터 로드
- RAG의 첫 단계는 지식의 원천이 될 데이터를 로드하는 것입니다.
- "Attention Is All You Need" 논문 PDF를 웹에서 직접 로드하겠습니다.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

pdf_url = "https://arxiv.org/pdf/1706.03762.pdf"
loader = PyPDFLoader(pdf_url)
docs = loader.load()

In [ ]:
print(f"'{docs[0].metadata['source']}' 에서 {len(docs)}개의 페이지를 로드했습니다.")
print("\n--- 첫 페이지 내용 샘플 ---")
print(docs[0].page_content[:500])
print("-" * 20)

## 3. 텍스트 전처리 (분할)
- LLM이 한 번에 처리할 수 있는 컨텍스트 길이는 제한되어 있습니다.
- 따라서 긴 문서를 의미 있는 단위의 작은 조각(Chunk)으로 분할해야 합니다.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=200  # 각 청크의 최대 크기  # 청크 간의 중복되는 글자 수
)
split_docs = text_splitter.split_documents(docs)

In [ ]:
print(f"총 {len(docs)}개의 페이지가 {len(split_docs)}개의 청크로 분할되었습니다.")
print("\n--- 첫 번째 청크 내용 샘플 ---")
print(split_docs[0].page_content)
print("-" * 20)

## 4. 임베딩 및 벡터 스토어 구축
- 분할된 텍스트 청크들을 컴퓨터가 이해할 수 있는 숫자 벡터(임베딩)로 변환하고,
- 이 벡터들을 효율적으로 검색할 수 있는 벡터 스토어에 저장합니다.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# HuggingFace에서 제공하는 임베딩 모델을 로드합니다.
# 'all-MiniLM-L6-v2'는 작고 빠르면서도 성능이 좋은 범용 모델입니다.
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
# 텍스트 청크와 임베딩 모델을 사용하여 FAISS 벡터 스토어를 생성합니다.
# 이 과정에서 각 청크가 벡터로 변환되어 인덱싱됩니다. (시간이 조금 걸릴 수 있습니다)
print("벡터 스토어를 생성하는 중입니다...")
vectorstore = FAISS.from_documents(split_docs, embedding_model)
print("벡터 스토어 생성이 완료되었습니다.")

In [ ]:
# 벡터 스토어에서 유사도 검색을 테스트해볼 수 있습니다.
retriever = vectorstore.as_retriever()
retrieved_docs = retriever.invoke("What is multi-head attention?")
print("\n--- 'multi-head attention' 검색 결과 (상위 1개) ---")
print(retrieved_docs[0].page_content)
print("-" * 20)

## 5. RAG Tool 생성
- 이제 위에서 만든 retriever를 Agent가 사용할 수 있는 도구로 만들어야 합니다.
- LangChain의 `create_retriever_tool` 함수를 사용하면 매우 간단하게 만들 수 있습니다.

In [ ]:
from langchain.tools.retriever import create_retriever_tool

# retriever_tool을 생성합니다.
# 이름과 설명을 명확하게 작성하는 것이 매우 중요합니다.
# Agent는 이 설명을 보고 언제 이 도구를 사용해야 할지 결정합니다.
retriever_tool = create_retriever_tool(
    retriever,
    "attention_is_all_you_need_search",
    "Searches and returns information about the 'Attention Is All You Need' paper. Use this tool for any questions about this specific paper, such as its authors, abstract, methodology like self-attention, multi-head attention, transformers, etc.",
)

## 6. Tavily Tool 생성

In [ ]:
from langchain_tavily import TavilySearch

# 지난 실습 파일들을 참고해서 적어보세요.

## 7. Agent 연동 및 테스트
- RAG Tool과 함께 웹 검색 도구도 Agent에게 제공하여,
- 질문의 종류에 따라 적절한 도구를 선택하는지 확인해 보겠습니다.

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

# LLM 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [ ]:
# 사용할 도구 목록
tools = [
    # [Your code]
]

In [ ]:
# Agent 생성
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(
    # [Your code]
)

agent_executor = AgentExecutor(
    # [Your code],
    verbose=True
)

## 8. 예시를 통한 결과 학인

In [ ]:
# 1. 논문 내용에 대한 질문 (RAG Tool을 사용해야 함)
print("\n--- Agent 실행 예시 1 (논문 관련 질문) ---")
question1 = "'Attention Is All You Need' 논문에서 제안하는 트랜스포머 모델의 주요 구성 요소는 무엇인가요?"

response1 = agent_executor.invoke({"input": question1})
print("\n[최종 답변]:", response1["output"])

In [ ]:
# 2. 일반적인 웹 검색 질문 (Tavily Tool을 사용해야 함)
print("\n--- Agent 실행 예시 2 (일반 상식 질문) ---")
question2 = "요즘 가장 인기 있는 프론트엔드 프레임워크는 무엇인가요?"
response2 = agent_executor.invoke({"input": question2})
print("\n[최종 답변]:", response2["output"])

In [ ]:
# 3. 논문 내용과 외부 지식이 모두 필요한 복합 질문
print("\n--- Agent 실행 예시 3 (복합 질문) ---")
question3 = "'Attention Is All You Need' 논문의 저자 중 한 명인 Ashish Vaswani의 다른 연구 활동에 대해 알려주세요."
# 이 질문에 답하기 위해 Agent는 먼저 논문에서 저자 목록을 찾고 (RAG Tool),
# 그 다음 특정 저자에 대한 정보를 웹에서 검색해야 합니다 (Tavily Tool).
response3 = agent_executor.invoke({"input": question3})
print("\n[최종 답변]:", response3["output"])